In [1]:
import scipy.io
import os
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
import Butterworth_filtering

from BW_metric import BW_dist
from AI_metric import AI_dist
from manifold_project_toolbox import F_dist
from manifold_project_toolbox import bw_projection_mean
from manifold_project_toolbox import ai_projection_mean
from manifold_project_toolbox import x2corr

from load_BCI_IV_IIa import import_data

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # enable multiple output in one cell

In [2]:
def MDM_dist(A, B, method):
    if method == "BW":
        output = BW_dist(A,B)
    elif method == "AI":
        output = AI_dist(A,B)
    elif method == "Euc":
        output = F_dist(A,B)
    return output

def MDM_mean(x, eps, method, verbose=False):
    # input: psd matrices [N,n,n]
    # output: mean matrix [n,n]
    if method == "BW":
        output = bw_projection_mean(x, eps, verbose=verbose)
    elif method == "AI":
        output = ai_projection_mean(x, eps, verbose=verbose)
    elif method == "Euc":
        output = np.mean(x, axis=0) # arithmetic mean
    return output

In [3]:
# BCI IV IIa
subject = 9
data, label = import_data("A0"+str(subject)+"T")
data_filtered = Butterworth_filtering.filter_all(data, order=5, lowcut=8, highcut=30, fs=250)

# left hand (class 1) vs. right hand (class 2)
idx = np.where(np.isin(label, [1, 2]))[0]
sample = x2corr(data_filtered)
sample = sample[idx]
label = label[idx]

sample.shape
label.shape

(144, 22, 22)

(144,)

In [4]:
# parameter setting
method = "AI"
repetition = 10 # number of reps for cv
k = 6 # k-fold cv

accuracy = np.zeros((repetition, k), dtype = np.float32)
execution_time = np.zeros((repetition, k), dtype = np.float32)

for rep_idx in range(repetition):

    ##### k-fold split
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=rep_idx+22)
    x_train_kfold = []
    y_train_kfold = []
    x_test_kfold = []
    y_test_kfold = []
    for train_idx, test_idx in kfold.split(sample, label):
        x_train_kfold.append(sample[train_idx])
        y_train_kfold.append(label[train_idx])
        x_test_kfold.append(sample[test_idx])
        y_test_kfold.append(label[test_idx])

    ##### MDM
    for fold_idx in range(k):
        # choose fold
        x_train = x_train_kfold[fold_idx]
        y_train = y_train_kfold[fold_idx]
        x_test = x_test_kfold[fold_idx]
        y_test = y_test_kfold[fold_idx]
        # compute mean of each class
        start_time = time.time() # timer starts
        mean1 = MDM_mean(x_train[np.where(y_train==1)], 0.01, method, verbose=True)
        mean2 = MDM_mean(x_train[np.where(y_train==2)], 0.01, method, verbose=True)
        # assign each testing sample to the nearest class mean
        y_pred = np.zeros(x_test.shape[0], dtype = np.float32)
        for test_idx in range(x_test.shape[0]):
            dist1 = MDM_dist(mean1, x_test[test_idx], method)
            dist2 = MDM_dist(mean2, x_test[test_idx], method)
            y_pred[test_idx] = 1 if dist1<dist2 else 2
        end_time = time.time() # timer stops
        # record accuracy and execution time
        accuracy[rep_idx,fold_idx] = sum(y_pred==y_test)/len(y_test)
        execution_time[rep_idx,fold_idx] = end_time - start_time
        
        print("Fold " + str(fold_idx+1) + " completed.")
    print("Repetition " + str(rep_idx+1) + " completed.")

logm result may be inaccurate, approximate err = 2.0857145579712465e-06
logm result may be inaccurate, approximate err = 2.8473848046692585e-06
logm result may be inaccurate, approximate err = 1.8015599884408082e-06
logm result may be inaccurate, approximate err = 3.1608915770662633e-06
logm result may be inaccurate, approximate err = 2.001180677106433e-06
logm result may be inaccurate, approximate err = 1.305472668648221e-06
logm result may be inaccurate, approximate err = 2.3892352965400496e-06
logm result may be inaccurate, approximate err = 2.196891159763332e-06
logm result may be inaccurate, approximate err = 2.3804697570078115e-06
logm result may be inaccurate, approximate err = 1.8969185553301839e-06
logm result may be inaccurate, approximate err = 1.6542598580415305e-06
logm result may be inaccurate, approximate err = 2.3101311549075677e-06
logm result may be inaccurate, approximate err = 1.5333027051756526e-06
logm result may be inaccurate, approximate err = 3.046135301330195e

In [5]:
np.mean(accuracy)
np.mean(execution_time)

1.0

1.1576902

In [6]:
rows = ['Rep{}'.format(i+1) for i in list(range(repetition))]
cols = ['Fold{}'.format(i+1) for i in list(range(k))]

accuracy_df = pd.DataFrame(accuracy, index=rows, columns=cols)
execution_time_df = pd.DataFrame(execution_time, index=rows, columns=cols)

In [7]:
save_dir = "D:/Projects/BW Revision/Results/BCI IV IIa/"
# accuracy_df.to_csv(save_dir+"A0"+str(subject)+"T"+"_accuracy_"+method+".csv")
execution_time_df.to_csv(save_dir+"Execution Time/"+"A0"+str(subject)+"T"+"_execution_time_"+method+".csv")